# Evaluation notebook

This file is part of a decoder for small topological surface and color codes, and potentially other stabilizer codes, when encoding a single logical qubit. The decoder is based on a combination of recurrent and feedforward neural networks.

Copyright (c) 2018, Paul Baireuther<br>
All rights reserved.

# Parameters to be set by user

In [ ]:
# # # FILES AND DIRECTORIES # # #

# Set path to source files of this project
src_path =  # "../src/"

# Set path to the checkpoints of the trained decoder
checkpoint_path =  # "../checkpoints/dist3/"

# Set path to the databases that contain the datasets
data_path =  # "../data/color_666_dist_3/"

# List of tuples with test database filenames and corresponding physical error rates
fnames = # [("colorcode_distance_3_test_p_0.002__canonical.db", 0.002),
#           ("colorcode_distance_3_test_p_0.003__canonical.db", 0.003),
#           ("colorcode_distance_3_test_p_0.004__canonical.db", 0.004),
#           ("colorcode_distance_3_test_p_0.006__canonical.db", 0.006),
#           ("colorcode_distance_3_test_p_0.010__canonical.db", 0.010),
#           ("colorcode_distance_3_test_p_0.016__canonical.db", 0.016),
#           ("colorcode_distance_3_test_p_0.025__canonical.db", 0.025),
#           ("colorcode_distance_3_test_p_0.040__canonical.db", 0.040),
#           ("colorcode_distance_3_test_p_0.063__canonical.db", 0.063),
#           ("colorcode_distance_3_test_p_0.100__canonical.db", 0.100),
#           ("colorcode_distance_3_test_p_0.160__canonical.db", 0.160),
#           ("colorcode_distance_3_test_p_0.250__canonical.db", 0.250)]

In [ ]:
# # # PARAMETERS # # #

# Network (parameters for 6-6-6 color code with distance 3)
code_distance = 3  # distance of the QEC code
dim_syndr = 12  # dimension of the syndrome increment vectors
dim_fsyndr = 3  # dimension of the final syndrome increment vectors
network_size = 32  # size of the LSTM's internal states and number of the the FF layers' neurons

# Network (parameters for 6-6-6 color code with distance 5)
# code_distance = 5  # distance of the QEC code
# dim_syndr = 36  # dimension of the syndrome increment vectors
# dim_fsyndr = 9  # dimension of the final syndrome increment vectors
# network_size = 64  # size of the LSTM's internal states and number of the the FF layers' neurons

# Network (parameters for 6-6-6 color code with distance 7)
# code_distance = 7  # distance of the QEC code
# dim_syndr = 72  # dimension of the syndrome increment vectors
# dim_fsyndr = 18  # dimension of the final syndrome increment vectors
# network_size = 128  # size of the LSTM's internal states and number of the the FF layers' neurons

# Specify max length of sequences in test datasets
max_len_test_sequences = 10000

# Evaluation parameters
# The number of datapoints per database is given by N = n_batches * batch_size
# In order to reduce memory costs at the expense of potentially slower evaluation
# decrease batch_size while keeping N constant
n_batches = 40  # number of batches ,
batch_size = 25  # number of sequences per batch

# Bootstrapping to get error estimates
bootstrap = True  # if True, error bars are calculated using bootstrapping
n_sampling = 5000  # number of draws during bootstrapping

# Main part of the notebook

In [ ]:
# # # LIBRARIES # # #

# Third party libraries
import os
import sys
import numpy as np

# This project
sys.path.insert(0, src_path);
import decoder as dec
import database_io as qec_db
import qec_functions as fcts

In [ ]:
# First we determine the best instance of the decoder from checkpoint_path/plog_history.dat

# Check if the checkpoint_path and contains a model folder exists
assert(os.stat(checkpoint_path + "model/"))

# Determine best model from plog_history.dat
plog_history = np.loadtxt(checkpoint_path + "plog_history.dat").reshape([-1, 2])
best_model_index = int(sorted(plog_history, key=lambda x: x[1])[0][0])
print("The best model was after epoch", best_model_index, 
      "with logical error rate", sorted(plog_history, key=lambda x: x[1])[0][1],
      "on the validation dataset.")

# Check if model file exists
assert(os.stat(checkpoint_path + "model/model-" + str(best_model_index) + ".index"))
assert(os.stat(checkpoint_path + "model/model-" + str(best_model_index) + ".meta"))

In [ ]:
# Initialize decoder and load the chosen model
net = dec.Decoder(code_distance=code_distance,
                  dim_syndr=dim_syndr,
                  dim_fsyndr=dim_fsyndr,
                  lstm_iss=[network_size, network_size],
                  ff_layer_sizes=[network_size],
                  checkpoint_path=checkpoint_path,
                  keep_prob=1.0,
                  aux_loss_factor=0.5,
                  l2_prefactor=10**(-5))

# Start TensorFlow session
net.start_session()

# Load the chosen model
net.load_network("model-"+str(best_model_index))

In [ ]:
# For all test databases in the list
for fn, p_phys in fnames[::-1]:

    # First we load the test database
    test_db_fname = data_path + fn
    db = qec_db.Data(training_fname=None,
                     validation_fname=None,
                     test_fname=test_db_fname)
    c = db.db_dict["test"].cursor()
    print("physical error rate in percent:", p_phys * 100)
    
    # We then take a single datapoint to learn about the maximum number
    # of steps in the test database
    test_batches = db.gen_batches(n_batches=1,
                                  batch_size=1,
                                  db_type="test",
                                  len_min=1,
                                  len_max=max_len_test_sequences,
                                  len_buffered=None)
    batch = next(test_batches)
    max_len = len(batch[1][0])
    
    # Based on this we choose a step size
    if max_len > 25:
        step = int(max_len / 25)
    else:
        step = 1
    step_l = [el for el in list(range(step, max_len + step, step)) if el <= max_len]
    
    # And then load all the data according to the chosen steps
    test_batches = db.gen_batches(n_batches=n_batches,
                                  batch_size=batch_size, 
                                  db_type="test",
                                  step_list = step_l,
                                  len_min=1,
                                  len_max=max_len,
                                  len_buffered=None)
    
    # Now we evaluate the decoder on the test database and calculate
    # logical error rate and error bars (optional)
    ret = net.test_net(test_batches)
    res_dict = fcts.calc_stats(data=ret, 
                               bootstrap=bootstrap, 
                               n_sampling=n_sampling, 
                               p0=(0.001, 0.1),
                               lower_bounds=(-0.1, -9999),
                               upper_bounds=(0.4, 9999),
                               verbose=True)
    res_dict['phys'] = p_phys
    
    # Finally, we plot the fidelity curve ...
    fcts.plot_fids(stats_dict=res_dict,
                   fmin=min(res_dict['fids']),
                   error_bars=bootstrap)
    
    # ... and print the logical error rate
    print("logical error rate in percent", res_dict['plog']*100)
    print("\n\n")